# Demand side flexibility


In [2]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")
#Pkg.add("NLopt")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [3]:
using CSV, DataFrames, JuMP, Gurobi

# Read the CSV data

In [4]:
SCALE_MWH = 1000
SCALE_DOLLAR = 1000;

In [5]:
fixed_cost = CSV.read("fixed_cost_2.csv",DataFrame) ./ SCALE_DOLLAR

Row,Fixed Cost,Fixed OM cost
,Float64,Float64
1,543.5,21.7531
2,666.821,28.7795
3,1110.5,37.5
4,3415.0,103.5
5,434.537,0.0
6,186.085,23.3723


In [6]:
variable_cost = CSV.read("variable_cost_2.csv",DataFrame) ./ SCALE_DOLLAR

Row,Variable OM,Ramp,Start
,Float64,Float64,Float64
1,0.0,0.0,0.0
2,0.0,0.0,0.002
3,0.00589,0.006,0.035
4,0.011176,0.015,0.07
5,0.0,0.001,0.0
6,0.0,0.0,0.0


In [7]:
co2_intensity = CSV.read("co2_intensity.csv",DataFrame)

Row,CO2 Intensity
,Int64
1,0
2,0
3,452
4,965
5,0
6,0


In [8]:
P_max_s = 95897829.43 / SCALE_MWH
P_max_w = 11158008 / SCALE_MWH;
P_max_s, P_max_w

(95897.82943000001, 11158.008)

In [9]:
Availability_matrix = CSV.read("Availability.csv", DataFrame);
println(size(Availability_matrix))

(8760, 5)


In [10]:
demand = CSV.read("demand.csv", DataFrame) ./ SCALE_MWH;
println(size(demand))
first(demand, 5)

(8760, 4)


Row,Commercial,Industrial,Residential,Transportation
,Float64,Float64,Float64,Float64
1,177.72,109.564,169.055,55.3949
2,174.776,108.33,164.425,37.6361
3,173.069,102.035,165.18,33.1084
4,171.107,96.909,168.945,29.3248
5,171.561,94.8884,173.68,23.6573


In [11]:
ramp_limits = CSV.read("ramp_limits.csv", DataFrame)

Row,Ramp_limit
,Float64
1,1.0
2,0.1
3,0.025
4,0.03
5,1.0
6,1.0


In [12]:
curtailment_costs = [0, 2, 0, 0, 0, 0] ./ SCALE_DOLLAR

6-element Vector{Float64}:
 0.0
 0.002
 0.0
 0.0
 0.0
 0.0

In [13]:
hrs = 4000
hre = 5000
sel_hrs = hrs:hre
println(sel_hrs)
techs = 4
n_eu = 4
# pad year by 1 day
add_h = 24

#Change to consider only RE + Battery
C_FC = fixed_cost[[1,2,5],1]
C_FOM = fixed_cost[[1,2,5],2]
E_CO2 = co2_intensity[[1,2,5],1]
C_VOM = variable_cost[[1,2,5],1]
C_ramp = variable_cost[[1,2,5],2]
C_start = variable_cost[[1,2,5],3]
C_curt = curtailment_costs[[1,2,5]]

D_b = demand[sel_hrs, 1:4] # Baseline demand with no flex
println(size(D_b))
D_b = vcat(reverse(reverse(demand)[1:add_h, 1:n_eu]), D_b, D_b[1:24, 1:n_eu])
println(size(D_b))
total_demand = sum(eachcol(D_b))
X_D = total_demand

A_r = ramp_limits[1:techs, 1]

A_s = Availability_matrix[sel_hrs,1:techs]
A_s = vcat(reverse(reverse(Availability_matrix)[1:add_h, 1:n_eu]), A_s, A_s[1:24, 1:n_eu])
println(size(A_s))
# over time, opt each year with worst day for availability and highest demand?

4000:5000
(1001, 4)
(1049, 4)
(1049, 4)


# Constant

In [14]:
eta = 0.8; # Efficiency of the battery
CO2_price = 100; # price of the CO2 per ton
H = 4;

# Creating the model

List of variable 

In [15]:
Power_generation_ls = 1:3 # Solar, Wind, Battery

fossil_fuel_ls = 1:2 # CCGT, Coal

nb_hours_ls = 1:size(X_D,1)

H = 2

2

In [24]:
model = Model(Gurobi.Optimizer)
println("Declare Model")
set_optimizer_attribute(model, "NonConvex", 2)
#model = Model(NLopt.Optimizer)
#set_optimizer_attribute(model, "algorithm", :LD_MMA)

#The installed power of the mix
@variable(model, P[Power_generation_ls] >= 0)


# Installed capacity renewables < max capacity
println("Renewable maximum installed capacity constraint")
@constraint(model, Con_sgen, P[1] <= P_max_s*10);
@constraint(model, Con_wgen, P[2] <= P_max_w*10);

@constraint(model, Power_battery_lb, P[3]>= 1000)
@constraint(model, Power_battery_ub, P[3]<= 10000)
#Generation of solar, wind, coal, CCGT
@variable(model, X_gen[nb_hours_ls, Power_generation_ls ] >= 0)


@variable(model, X_soc[nb_hours_ls]>=0)

println("Objective function")
CO2_price = 100
# The cost of the mix assuming a certain carbon cost
#@objective(model, Min, sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:size(P,1)) +
#sum(sum((C_VOM[j] + CO2_price * E_CO2[j])* X_gen[t,j] for j in 1:size(P,1)) for t in 1:size(X_gen,1)))


@objective(model, Min, 
    (
        sum((C_FC[i] + C_FOM[i])*P[i] for i in Power_generation_ls) +
        sum(sum((C_VOM[i] + CO2_price * E_CO2[i])* X_gen[t, i] for i in Power_generation_ls) for t in nb_hours_ls)
));

# For each tech, at each hour, usage cannot exceed available capacity
println("Availability & installed power constraint")
@constraint(model, Max_production[i in Power_generation_ls, t in nb_hours_ls], X_gen[t, i] <= P[i]*A_s[t, i])

# Supply meets demand at each hours:
println("Supply meets demand constraint")
@constraint(model, Supply_demand[t in nb_hours_ls], sum(X_gen[t,i] for i in Power_generation_ls) >= X_D[t]);


Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-07
Declare Model
Set parameter NonConvex to value 2
Renewable maximum installed capacity constraint
Objective function
Availability & installed power constraint
Supply meets demand constraint


![Alt text](image.png)

Variable to indicate if $x_{g,t}<x_{d,t}$

In [25]:
M = 10000000000000


@variable(model, Z_X_gen_lower_X_D[nb_hours_ls], Bin)

@constraint(model, Z_X_gen_lower_X_D_1[t in nb_hours_ls], X_D[t] - sum(X_gen[t,i] for i in 1:2)  <= M*Z_X_gen_lower_X_D[t])
@constraint(model, Z_X_gen_lower_X_D_2[t in nb_hours_ls], -X_D[t] + sum(X_gen[t,i] for i in 1:2) <= M*(1-Z_X_gen_lower_X_D[t]));


Create the variable X_min = min(X_1, X_2, X_3)

In [26]:
# X_1, X_2, X_3 = [eta*(sum(X_gen[t,i] for i in Power_generation_ls)- X_D[t]) , eta*P3], (1- X_soc[t])*P[5]* H]
@variable(model, z1[t in nb_hours_ls], Bin)
@variable(model, z2[t in nb_hours_ls], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("Z_1 expression")
@constraint(model, z_1_1[t in nb_hours_ls], eta*P[3] - eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t])   <=  M*z1[t])
@constraint(model, z_1_2[t in nb_hours_ls], -eta*P[3] + eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t]) <= M*(1-z1[t]))


#Minimum between X_1 and X_2
@variable(model, min_X_1_X_2[nb_hours_ls])

println("Min X_1, X_2 expression")
@constraint(model, X_1_X_2_lower_1[t in nb_hours_ls], min_X_1_X_2[t] <= eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t]))
@constraint(model, X_1_X_2_lower_2[t in nb_hours_ls], min_X_1_X_2[t] <= eta*P[3])
@constraint(model, X_1_X_2_higher_1[t in nb_hours_ls], min_X_1_X_2[t] >= eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t]) - M*(1-z1[t]))
@constraint(model, X_1_X_2_higher_2[t in nb_hours_ls], min_X_1_X_2[t] >= eta*P[3] - M*z1[t])


# z_2 equal 1 if min(X_1, X_2) is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
@constraint(model, z_2_1[t in nb_hours_ls], (1- X_soc[t])* P[3] * H - min_X_1_X_2[t] <=  M*z2[t])
@constraint(model, z_2_2[t in nb_hours_ls], - (1- X_soc[t]) * P[3] * H + min_X_1_X_2[t] <= M*(1-z2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
@variable(model, min_X_1_X_2_X_3[nb_hours_ls])

@constraint(model, X_1_X_2_X_3_lower_1[t in nb_hours_ls], min_X_1_X_2_X_3[t] <= min_X_1_X_2[t])
@constraint(model, X_1_X_2_X_3_lower_2[t in nb_hours_ls], min_X_1_X_2_X_3[t] <= (1- X_soc[t])*P[3]* H)
@constraint(model, X_1_X_2_X_3_higher_1[t in nb_hours_ls], min_X_1_X_2_X_3[t] >= min_X_1_X_2[t] - M*(1-z2[t]))
@constraint(model, X_1_X_2_X_3_higher_2[t in nb_hours_ls], min_X_1_X_2_X_3[t] >= (1- X_soc[t])*P[3]* H - M*z2[t]);



Z_1 expression
Min X_1, X_2 expression
 Take the min of min(X_1, X_2) and X_3
 Take the overall min


Expression of X_ch

In [ ]:
# Need to put this conditions above
@variable(model, Z_X_gen_greater_X_D[nb_hours_ls], Bin)

@constraint(model, Z_X_gen_greater_X_D_1[t in nb_hours_ls], sum(X_gen[t,i] for i in 1:2) - X_D[t] <= M*Z_X_gen_greater_X_D[t])
@constraint(model, Z_X_gen_greater_X_D_2[t in nb_hours_ls], -sum(X_gen[t,i] for i in 1:2) + X_D[t] <= M*(1-Z_X_gen_greater_X_D[t]))

#Expression of X_ch
@constraint(model, X_ch_expression[t in nb_hours_ls], X_ch[t]  == Z_X_gen_greater_X_D[t]*min_X_1_X_2_X_3[t]);

Expression of min of [X_gen[t]- X_D[t],  P[5], X_soc[t]*P3]* H]

In [ ]:
# X_1, X_2, X_3 = [X_gen[t]- X_D[t]) , P3], X_soc[t])*P3]* H]
#@variable(model, y1[nb_hours_ls], Bin)
#@variable(model, y2[nb_hours_ls], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
#println("Y_1 expression")
#@constraint(model, y_1_1[t in nb_hours_ls], P[3] - (sum(X_gen[t,i] for i in 1:2)- X_D[t])   <=  M*y1[t])
#@constraint(model, y_1_2[t in nb_hours_ls], -P[3] + (sum(X_gen[t,i] for i in 1:2)- X_D[t]) <= M*(1-y1[t]))


#Minimum between X_1 and X_2
#@variable(model, min_extra_prod_P_5[nb_hours_ls])

println("Min X_1, X_2 expression")
#@constraint(model, min_extra_prod_P_5_lower_1[t in nb_hours_ls], min_extra_prod_P_5[t] <= sum(X_gen[t,i] for i in 1:2)- X_D[t])
#@constraint(model, min_extra_prod_P_5_lower_2[t in nb_hours_ls], min_extra_prod_P_5[t] <= P[3])
#@constraint(model, min_extra_prod_P_5_higher_1[t in nb_hours_ls], min_extra_prod_P_5[t] >= (sum(X_gen[t,i] for i in 1:2)- X_D[t]) - M*(1-y1[t]))
#@constraint(model, min_extra_prod_P_5_higher_2[t in nb_hours_ls], min_extra_prod_P_5[t] >= P[3] - M*y1[t])


# z_2 equal 1 if min(X_1, X_2) is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
#@constraint(model, y_2_1[t in nb_hours_ls], X_soc[t]* P[3] * H - min_extra_prod_P_5[t] <=  M*y2[t])
#@constraint(model, y_2_2[t in nb_hours_ls], - X_soc[t] * P[3] * H + min_extra_prod_P_5[t] <= M*(1-y2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
#@variable(model, min_extra_prod_P5_soc[nb_hours_ls])

#@constraint(model, min_extra_prod_P5_soc_lower_1[t in nb_hours_ls], min_extra_prod_P5_soc[t] <= min_extra_prod_P_5[t])
#@constraint(model, min_extra_prod_P5_soc_lower_2[t in nb_hours_ls], min_extra_prod_P5_soc[t] <= X_soc[t]*P[3]* H)
#@constraint(model, min_extra_prod_P5_soc_higher_1[t in nb_hours_ls], min_extra_prod_P5_soc[t] >= min_extra_prod_P_5[t] - M*(1-y2[t]))
#@constraint(model, min_extra_prod_P5_soc_higher_2[t in nb_hours_ls], min_extra_prod_P5_soc[t] >= X_soc[t]*P[3]* H - M*y2[t]);



Y_1 expression
Min X_1, X_2 expression
 Take the min of min(X_1, X_2) and X_3
 Take the overall min


Expression of X_dch

In [ ]:

#Expression of X_dch
# issue with this constraint
#@constraint(model, X_dch_expression[t in nb_hours_ls], X_gen[t,3]  == Z_X_gen_lower_X_D[t]*min_extra_prod_P5_soc[t]);

Expression of state of charge

In [27]:


#@constraint(model, SOC_t_1, X_soc[1] == 1)

#@constraint(model, SOC_t[t in nb_hours_ls[2:end]], (X_soc[t]-X_soc[t-1])*H*P[3] == (X_ch[t-1]- X_dch[t-1]));

@constraint(model, SOC_t[t in nb_hours_ls], X_soc[t]== 0.5);

In [28]:
optimize!(model);

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 15739 rows, 9444 columns and 42815 nonzeros
Model fingerprint: 0xeec528f7
Model has 4196 quadratic constraints
Variable types: 6297 continuous, 3147 integer (3147 binary)
Coefficient statistics:
  Matrix range     [2e-09, 1e+13]
  QMatrix range    [2e+00, 2e+00]
  QLMatrix range   [1e+00, 1e+13]
  Objective range  [4e+02, 7e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 1e+13]
  QRHS range       [1e+13, 1e+13]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2738 rows and 3756 columns
Presolve time: 0.86s
Presolved: 13001 rows, 5688 columns, 41070 nonzeros
Variable types: 3261 continuous, 2427 integer (2427 binary)

Root relaxation: objective 1.879155e+07, 2213 iterations, 0.01 s

In [ ]:
objective_value(model)

LoadError: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.

In [53]:
println(value.(P))
#println(value.(s1[:]))
println(value.(X_soc))



1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:3
And data, a 3-element Vector{Float64}:
     0.0
 26390.163638745926
  1000.0
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:1049
And data, a 1049-element Vector{Float64}:
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5

In [31]:
#println(value.(X_gen))
X_gen_matrix = value.(X_gen);




In [42]:
using Plots

@userplot StackedArea

# a simple "recipe" for Plots.jl to get stacked area plots
# usage: stackedarea(xvector, datamatrix, plotsoptions)
@recipe function f(pc::StackedArea)
    x, y = pc.args
    n = length(x)
    y = cumsum(y, dims=2)
    seriestype := :shape

    # create a filled polygon for each item
    for c=1:size(y,2)
        sx = vcat(x, reverse(x))
        sy = vcat(y[:,c], c==1 ? zeros(n) : reverse(y[:,c-1]))
        @series (sx, sy)
    end
end

a = X_gen_matrix[:,1]
b = X_gen_matrix[:,2]
c = X_gen_matrix[:,3]
sNames = ["a","b","c"]
x = nb_hours_ls

plotly()
stackedarea(x, [a b c], labels=reshape(sNames, (1,3)))
#genplot(x, X_D[:])

In [76]:
println(size(min_X_1_X_2_X_3))
Z_X_gen_lower_X_D_values = value.(Z_X_gen_lower_X_D[:])
X_minX3 = value.(min_X_1_X_2_X_3)
X_2 = value.(eta*(sum(X_gen[:,i] for i in 1:2)- X_D[:]))
X_1 = value.(eta*P[3]*(ones(size(X_soc))))
X_3 = value.((ones(size(X_soc))- X_soc[:])*P[3]* H );

ls_X_min = []

for i = X_minX3
    ls_X_min = [ls_X_min; i]
    #println(i)
end
println(ls_X_min)
plotly()
x = nb_hours_ls
plot(x, [ X_1 X_2 X_3], label=["XMIN" "X1" "X2" "X3"])
plot!(x, ls_X_min, label="Min", lc=:black, lw=2, ls=:dot)


(1049,)
Any[800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, -546.0193807400037, 800.0, -565.644028026004, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, -647.7593719439992, 800.0, 800.0, 800.0, -737.1035047200003, 800.0, -785.3589265600003, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, -587.5169070400007, 800.0, 800.0, 800.0, 800.0, -706.2554881600008, 800.0, -695.3356653600022, 800.0, 800.0, 800.0, -597.1403942399993, -539.6246088799999, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, -382.11062536600184, -417.6599086020042, 800.0, 800.0, -560.6672543999994, 800.0, 800.0, -658.4956838399994, -671.7692286399997, 800.0, 800.0, 800.0, 800.0, -667.0209267200007, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, -377.72810804800065, 0.0007638280000037412, -0.0009691359999806082, 800.0, 800.0, 800.0, -565.1272168600071, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 800.0, -800.0001794350001, 800.0, 800.0, 

In [ ]:
P_3_matrix = value(P[3])*ones(size(nb_hours_ls))

println(P_3_matrix)
println("X gen")
X_gen_non_battery = sum(X_gen_matrix[:,i] for i =1:2) - X_D[:]
println("X soc")
X_soc_charge = (ones(size(X_soc))- value.(X_soc))* value(P[3]) * H

println("X ch")
X_charge = value.(X_ch)

plot(x, [P_3_matrix X_gen_non_battery X_soc_charge X_charge], label=["P[3]" "X_gen Non batt" "X possible charge" "X_ch"])


[3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080

 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080

In [ ]:
println(value.(min_extra_prod_P5_soc))
println(value.(Z_X_gen_lower_X_D))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:1000
And data, a 1000-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.